In [ ]:
import os
import streamlit as st
import pandas as pd
from matplotlib import pyplot as plt
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime, timedelta
import seaborn as sns

env_file_path = 'D:/Projects/.env'
load_dotenv(env_file_path)
# Giving output file name
output_file_path = 'HOURLY.xlsx'
# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = R'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

##################### ACCESS ENV VARIABLES ######################
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_SERGELI")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

##################### PROCEDURE NAME ######################
procedure_name = os.getenv("HOURLY_SHORT")  # THIS IS HOURLY DATA GATHERING

##################### DATE - JANUARY ######################
CURRENT_MONTH = 1
CURRENT_YEAR = 2024
today_date = datetime.now().strftime('%d/%m/%Y')
tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%d/%m/%Y')
##################### CONNECTION STRING AND SQL QUERY ######################
# Construct the connection string
conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)

sql_query = f"""
    DECLARE @DateBegin DATE = ?;
    DECLARE @DateEnd DATE = ?;

    EXEC {procedure_name}
        @DateBegin = @DateBegin,
        @DateEnd = @DateBegin;
"""

#####################  EXECUTION  ######################
df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))
df.columns = [
    'DocumentType', 'Invoice Number', 'Goodid', 'Good', 'Manufacturer',
    'inn', 'ClientName', 'SalesManager', 'ClientMan', 'PaymentTerm',
    'BasePrice', 'SellingPrice', 'Quantity', 'DateEntered', 'BaseAmount', 'TotalAmount'
]
df = df[df['DocumentType'].isin(['Оптовая реализация', 'Финансовая скидка'])]
df = df[~(df['SalesManager'] == 'Бочкарева Альвина')]
df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientMan', right_on='ClientMan', how='left')

df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')
df['inn_temp'] = pd.to_numeric(df['inn'], errors='coerce')
types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
df = pd.merge(df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp', how='left')
df.fillna({'TYPE': 'ROZ'}, inplace=True)
df.loc[df['TYPE'] == 'ROZ', 'RegionType'] = df['Region']
df['OXVAT'] = df['inn'].map(df['inn'].value_counts())
categorical_columns = ['DocumentType', 'Good', 'Manufacturer', 'inn', 'ClientName', 'SalesManager', 'ClientMan',
                       'PaymentTerm', 'Region', 'RegionType', 'TYPE']

df[categorical_columns] = df[categorical_columns].astype('category')
columns_to_drop = [col for col in df.columns if col.endswith('_temp')]
df.drop(columns=columns_to_drop, inplace=True)


In [ ]:
gr_df = df.groupby(['TYPE', 'RegionType'], observed=False)['TotalAmount'].sum().reset_index()
unique_types = gr_df['TYPE'].unique()

In [33]:
gr_df = gr_df[gr_df['TotalAmount'] != 0]


In [34]:
gr_df

,TYPE,RegionType,TotalAmount
0,ROZ,Андижан,2.979916e+08
2,ROZ,Бухара,2.150854e+08
4,ROZ,Джизак,8.005293e+07
5,ROZ,Карши,4.755022e+08
7,ROZ,Коканд,1.370107e+08
8,ROZ,Навои,1.918754e+07
10,ROZ,Наманган,1.394167e+08
11,ROZ,Нукус,3.517162e+07
12,ROZ,Самарканд,5.287914e+08
13,ROZ,Термез,2.291366e+08
